In [1]:
# lets build sales data
import build_sales_db
build_sales_db.main()

✅ Created and seeded sales.db


# Data is already in database.

* We want rag to answer questions such as 
```
How many items did we sold last month

Which item got most revenue last month
```

In [2]:
# lets create llm
from langchain_google_vertexai import ChatVertexAI
model = ChatVertexAI(
    model_name="gemini-2.5-flash-lite",
)

C:\Users\DELL\AppData\Local\Temp\ipykernel_25604\3522371365.py:3: DeprecationWarning: Use [`ChatGoogleGenerativeAI`][langchain_google_genai.ChatGoogleGenerativeAI] instead.
  model = ChatVertexAI(
C:\Users\DELL\AppData\Local\Temp\ipykernel_25604\3522371365.py:3: LangChainDeprecationWarning: The class `ChatVertexAI` was deprecated in LangChain 3.2.0 and will be removed in 4.0.0. An updated version of the class exists in the `langchain-google-genai package and should be used instead. To use it run `pip install -U `langchain-google-genai` and import as `from `langchain_google_genai import ChatGoogleGenerativeAI``.
  model = ChatVertexAI(


In [ ]:
# Langchain support sql agents

from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_community.utilities import SQLDatabase
db = SQLDatabase.from_uri("sqlite:///sales.db")

db_tool_kit = SQLDatabaseToolkit(db=db, llm=model)


In [5]:
from langchain_community.agent_toolkits import create_sql_agent

agent = create_sql_agent(
    llm=model,
    toolkit=db_tool_kit,
    verbose=True,
)

In [6]:
agent

AgentExecutor(name='SQL Agent Executor', verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'tools': "sql_db_query - Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.\nsql_db_schema - Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3\nsql_db_list_tables - Input is an empty string, output is a comma-separated l

In [7]:
response = agent.invoke(
    {"input": "How many items did we sold last month"}
)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input:customers, order_items, orders, productsThe user is asking for the number of items sold last month. This information is likely in the `orders` and `order_items` tables. I should inspect the schema of these tables to understand how to calculate the total number of items sold.
Action: sql_db_schema
Action Input: orders, order_items
CREATE TABLE order_items (
	order_item_id INTEGER, 
	order_id INTEGER NOT NULL, 
	product_id INTEGER NOT NULL, 
	qty INTEGER NOT NULL, 
	unit_price REAL NOT NULL, 
	discount REAL NOT NULL, 
	PRIMARY KEY (order_item_id), 
	FOREIGN KEY(order_id) REFERENCES orders (order_id), 
	FOREIGN KEY(product_id) REFERENCES products (product_id)
)

/*
3 rows from order_items table:
order_item_id	order_id	product_id	qty	unit_price	discount
1	1	28	3	3516.32	0.15
2	2	42	5	510.81	0.15
3	2	18	4	39362.22	0.2
*/


CREATE TABLE orders (
	order_id INTEGER, 
	customer_id INTEGER NOT NULL, 
	order_date

In [8]:
response

{'input': 'How many items did we sold last month', 'output': '1745'}

In [9]:
response = agent.invoke(
    {"input": "Which item was sold the most last month"}
)




> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: customers, order_items, orders, productsThe user is asking about sales data from "last month", which suggests I need to look at tables related to orders and products. I should examine the schema for `orders` and `order_items` to understand how sales are recorded and if there's a timestamp for when orders were placed.
Action: sql_db_schema
Action Input: orders, order_items
CREATE TABLE order_items (
	order_item_id INTEGER, 
	order_id INTEGER NOT NULL, 
	product_id INTEGER NOT NULL, 
	qty INTEGER NOT NULL, 
	unit_price REAL NOT NULL, 
	discount REAL NOT NULL, 
	PRIMARY KEY (order_item_id), 
	FOREIGN KEY(order_id) REFERENCES orders (order_id), 
	FOREIGN KEY(product_id) REFERENCES products (product_id)
)

/*
3 rows from order_items table:
order_item_id	order_id	product_id	qty	unit_price	discount
1	1	28	3	3516.32	0.15
2	2	42	5	510.81	0.15
3	2	18	4	39362.22	0.2
*/


CREATE TABLE orders (
	order_id INTEGER, 

In [10]:
print(response)

{'input': 'Which item was sold the most last month', 'output': 'Product 031'}


In [11]:
response = agent.invoke(
    {"input": "Give me top 5 items sold by revenu in descending order with item name, count and total revenue generated"}
)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: customers, order_items, orders, productsThe user is asking for the top 5 items sold by revenue. This information is likely to be found in tables related to orders and products. I should inspect the schema of `order_items` and `products` tables to understand how to join them and extract the required information.
Action: sql_db_schema
Action Input: order_items, products
CREATE TABLE order_items (
	order_item_id INTEGER, 
	order_id INTEGER NOT NULL, 
	product_id INTEGER NOT NULL, 
	qty INTEGER NOT NULL, 
	unit_price REAL NOT NULL, 
	discount REAL NOT NULL, 
	PRIMARY KEY (order_item_id), 
	FOREIGN KEY(order_id) REFERENCES orders (order_id), 
	FOREIGN KEY(product_id) REFERENCES products (product_id)
)

/*
3 rows from order_items table:
order_item_id	order_id	product_id	qty	unit_price	discount
1	1	28	3	3516.32	0.15
2	2	42	5	510.81	0.15
3	2	18	4	39362.22	0.2
*/


CREATE TABLE products (
	product_id INTEGER, 

In [12]:
response

{'input': 'Give me top 5 items sold by revenu in descending order with item name, count and total revenue generated',
 'output': 'The top 5 items sold by revenue are:\n1. Product 018: Count = 377, Total Revenue = 12559527.948\n2. Product 053: Count = 352, Total Revenue = 11531437.777\n3. Product 001: Count = 365, Total Revenue = 11048054.2645\n4. Product 011: Count = 364, Total Revenue = 10949414.466500001\n5. Product 044: Count = 340, Total Revenue = 10136998.4285'}

In [13]:
print(response['output'])

The top 5 items sold by revenue are:
1. Product 018: Count = 377, Total Revenue = 12559527.948
2. Product 053: Count = 352, Total Revenue = 11531437.777
3. Product 001: Count = 365, Total Revenue = 11048054.2645
4. Product 011: Count = 364, Total Revenue = 10949414.466500001
5. Product 044: Count = 340, Total Revenue = 10136998.4285


# Lets try csv

In [14]:
from build_sales_csv import main as build_csv
build_csv()

✅ CSV files created:
 - customers.csv
 - products.csv
 - orders.csv
 - order_items.csv


In [15]:
import pandas as pd
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

In [17]:

products_df = pd.read_csv('products.csv')
orders_df = pd.read_csv('orders.csv')
order_items_df = pd.read_csv('order_items.csv')
customers_df = pd.read_csv('customers.csv')

In [22]:
agent = create_pandas_dataframe_agent(
    llm=model,
    df=[orders_df, order_items_df, customers_df],
    verbose=True,
    allow_dangerous_code=True
)

In [23]:
agent.invoke(
    {"input": "How many items did we sold last month"})



> Entering new AgentExecutor chain...
Thought: The question asks about the number of items sold last month. To answer this, I need to:
1. Determine what "last month" refers to in the context of the data. The `order_date` column in `df1` seems to be the relevant one. I should find the most recent order date in the dataset to establish a reference point for "last month".
2. Calculate the date range for "last month" based on the most recent order date.
3. Filter `df1` to include only orders placed within "last month".
4. Join the filtered `df1` with `df2` (which contains `order_item_id` and `qty`) on `order_id`.
5. Sum the `qty` column from the joined dataframe to get the total number of items sold.

Let's start by finding the most recent order date.
Action: python_repl_ast
Action Input: print(df1['order_date'].max())2026-02-03
The most recent order date is 2026-02-03. Therefore, "last month" refers to January 2026.

Now I need to:
1. Filter `df1` to include only orders placed in Januar

{'input': 'How many items did we sold last month', 'output': '1745'}

# Better approach to deal with this kind of data

* Use DuckDB